# Test for the Raw library.

This is a simple library helping with display of data in Json form.

Two classes: Nest supports hierarchical iteration needed by json, JsonSqlClient has simple sql read operations supporting Json and multiple rowsets.



#### tl;dr

a simple block of code to test rendering of a razor page with styles from a set of database queries.

In [1]:
// load packages - your local nuget folder may vary
#i "file:///n:/nix-nuget/"
#r "nuget:nix-library-raw"  
#r "nuget: Microsoft.Data.SqlClient, 5.1.5"
// you may want the razor clinteractive support too!
#r "nuget: RazorInteractive"

Restore sources file:///n:/nix-nuget/ Installed Packages Microsoft.Data.SqlClient, 5.1.5 nix-library-raw, 1.0.17 RazorInteractive, 1.2.0

Loading extensions from `C:\Users\NickMajor\.nuget\packages\razorinteractive\1.2.0\lib\net8.0\RazorInteractive.dll`

Renders the code block as Razor markup in dotnet-interactive notebooks. 
 This extension adds a new kernel that can render Razor markdown. 
 All C# and F# variables are available in the @Model property.

In [2]:
using Nix.Library.Raw;
string connectionString = System.Environment.GetEnvironmentVariable("Nix_DB_DG");
JsonSqlClient _sql = new JsonSqlClient(connectionString);

In [3]:
// samples using stored procedures
var x = _sql.JsonUsingStoredProcedure("[rpt].[ReportMethods_SelectByOrganization]", ("@OrganizationId", 1));
x.ToArray().Display();
var y = _sql.JsonUsingStoredProcedure("[rpt].[InsightKPIs]", ("@StartDate", "2022-02-22"), ("@EndDate", "2024-02-26"), ("@OrganizationId", 1));
foreach (var yy in y)
{
    yy.Display();
    Nest.Prettify(yy).Display();
}  
// a non-json instance...
var z = _sql.StringsUsingStoredProcedure("[rpt].[InsightKPIs]", true, ("@StartDate", "2022-02-22"), ("@EndDate", "2024-02-26"), ("@OrganizationId", 1));
z.ToArray().Display();



[ [{"Id":"7", "Name":"Custom", "StoredProcedureName":"client.ReportGeneration_Degreed", "Category":"Scheduled", "OrganizationId":"1", "Version":"", "IsStandard":"False"},{"Id":"47", "Name":"Skill Report", "StoredProcedureName":"rpt.ReportGeneration_Standard_SkillPlans_2020Q01", "Category":"Scheduled", "OrganizationId":"", "Version":"2020 Q1", "IsStandard":"True"},{"Id":"50", "Name":"Standard Report", "StoredProcedureName":"rpt.ReportGeneration_Standard_EmployeeId_2020Q02", "Category":"Scheduled", "OrganizationId":"", "Version":"2020 Q2", "IsStandard":"True"},{"Id":"54", "Name":"Skill Analytics Skill Report", "StoredProcedureName":"rpt.ReportGeneration_SkillAnalyticsSkillReport_2020Q3", "Category":"Scheduled", "OrganizationId":"", "Version":"2020 Q3", "IsStandard":"True"},{"Id":"61", "Name":"MockedData", "StoredProcedureName":"MockedData", "Category":"MockedData", "OrganizationId":"1", "Version":"", "IsStandard":"False"},{"Id":"1", "Name":"Finra Data", "StoredProcedureName":"FinraData", "Category":"ScheduledES", "OrganizationId":"", "Version":"", "IsStandard":"True"},{"Id":"51", "Name":"Career Mobility Report", "StoredProcedureName":"rpt.ReportGeneration_Standard_Opportunities_2020Q02", "Category":"Scheduled", "OrganizationId":"", "Version":"2020 Q2", "IsStandard":"True"}] ]

[{"Logins":"497"}]

[
  {
    "Logins": "497"
  }
]

[{"Views":"12684"}]

[
  {
    "Views": "12684"
  }
]

[{"Recommendations":"5605"}]

[
  {
    "Recommendations": "5605"
  }
]

index,value
0,[ Logins ]
1,[ 497 ]
2,[ Views ]
3,[ 12684 ]
4,[ Recommendations ]
5,[ 5605 ]


In [4]:
_sql.Dispose();

# Build a display: 

We can prepare all the pieces we need using C#, these will be available to the in-line Razor page in the @Model.

An alternative is to write the html to a file and use ps to launch it in the browser.

In [5]:
#!csharp
var output = new List<string>();
// this actually only needs to happen once...
Nest.ToFile(Nest.GetCss(new Nest.NestOptions { MaximumPixelWidth = 500, DarkMode=false}), "../.data/local-test.css");
output.Add(Nest.RenderHtmlScrolling(x[0])); // because we know there is only one result set.
// multiple result sets:
foreach (var dd in y)
{
    output.Add(Nest.RenderHtmlScrolling(dd));
}


In [6]:

#!razor
<html>
  <head>
    <link rel="stylesheet" href="./local-test.css">
  </head>
  <body>
      @foreach(var xx in @Model.output)
      {
        @Raw(xx)
        <hr>
      }
      Done
      <hr>
  </body>
</html>




Nest 
 1/7 2/7 3/7 4/7 5/7 6/7 7/7 Id 7 47 50 54 61 1 51 Name Custom Skill Report Standard Report Skill Analytics Skill Report MockedData Finra Data Career Mobility Report StoredProcedureName client.ReportGeneration_Degreed rpt.ReportGeneration_Standard_SkillPlans_2020Q01 rpt.ReportGeneration_Standard_EmployeeId_2020Q02 rpt.ReportGeneration_SkillAnalyticsSkillReport_2020Q3 MockedData FinraData rpt.ReportGeneration_Standard_Opportunities_2020Q02 Category Scheduled Scheduled Scheduled Scheduled MockedData ScheduledES Scheduled OrganizationId 1 1 Version 2020 Q1 2020 Q2 2020 Q3 2020 Q2 IsStandard False True True True False True True 
 Nest 
 1/1 Logins 497 
 Nest 
 1/1 Views 12684 
 Nest 
 1/1 Recommendations 5605 
 Done